In [153]:
# coding: utf-8
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# get_ipython().magic('matplotlib inline')
from nltk import word_tokenize
import re

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix, precision_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression

In [5]:
lines = pd.read_csv('All-seasons.csv', sep=',',
                           names=["season", "episode", "character", "line"])

In [7]:
cols_to_keep = ['character', 'line']
data = lines[cols_to_keep]

In [49]:
characters = ['Cartman', 'Kyle', 'Kenny', 'Stan']
main_data = data[data['character'].isin(characters)]

In [62]:
main_data = main_data.dropna()

In [64]:
length = []
character = []
lines = []
for m in main_data['line']:
    m = m.strip()
    m = m.lower()
    words = word_tokenize(m)
    length.append(len(words))
    lines.append(m)
for c in main_data['character']:
    character.append(c)

In [70]:
new_data = pd.DataFrame(
    {'character': character,
     'length': length,
     'line': lines
    })

In [67]:
print(new_data.groupby('character').describe())

                      length
character                   
Cartman   count  9774.000000
          mean     17.663802
          std      19.026016
          min       1.000000
          25%       7.000000
          50%      12.000000
          75%      22.000000
          max     351.000000
Kenny     count   881.000000
          mean      8.861521
          std       7.736653
          min       2.000000
          25%       4.000000
          50%       7.000000
          75%      11.000000
          max     144.000000
Kyle      count  7099.000000
          mean     11.989294
          std      11.124612
          min       1.000000
          25%       5.000000
          50%       9.000000
          75%      15.000000
          max     159.000000
Stan      count  7680.000000
          mean     12.042578
          std      11.029203
          min       1.000000
          25%       5.000000
          50%       9.000000
          75%      15.000000
          max     177.000000


In [55]:
def tokenize(text):
    text = text.lower()
    return word_tokenize(text)

In [72]:
bow = CountVectorizer(tokenizer=tokenize, stop_words='english')
bowed = bow.fit_transform(new_data['line'])

In [74]:
def split_into_lemmas(message):
    message = message.lower()
    return word_tokenize(message)
bow_transformer = CountVectorizer(analyzer=split_into_lemmas).fit(new_data['line'])

In [79]:
lines_bow = bow_transformer.transform(new_data['line'])

In [83]:
tfidf_transformer = TfidfTransformer().fit(lines_bow)
lines_tfidf = tfidf_transformer.transform(lines_bow)

In [104]:
X_train, X_test, y_train, y_test = train_test_split(lines_bow, new_data['character'], test_size=0.2)

In [105]:
# НАИВНЫЙ БАЙЕС

In [106]:
character_detector = MultinomialNB().fit(X_train, y_train)

In [ ]:
#И accuracy и precision средние.

In [144]:
y_pred_nb = character_detector.predict(X_test)
print(classification_report(y_test, y_pred_nb))
print(confusion_matrix(y_test, y_pred_nb))

             precision    recall  f1-score   support

    Cartman       0.58      0.71      0.63      1953
      Kenny       0.92      0.74      0.82       157
       Kyle       0.47      0.34      0.40      1440
       Stan       0.48      0.47      0.47      1537

avg / total       0.53      0.53      0.52      5087

[[1380    4  252  317]
 [  32  116    7    2]
 [ 475    4  494  467]
 [ 511    2  305  719]]


In [ ]:
#У Кенни высокий precision и неплохой recall, однако у него намного меньше фраз чем у других персонажей 
# и у других персонажей оба результата хуже. Хуже всего классификатор распознает Кайла. Примерно столько же реплик, сколько
# классификатор приписывает самому Кайлу, он приписывает еще и Кармену и Стэну.

In [ ]:
# ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ

In [145]:
lr = LogisticRegression(class_weight='balanced')
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print(classification_report(y_test, y_pred_lr))
print(confusion_matrix(y_test, y_pred_lr))

             precision    recall  f1-score   support

    Cartman       0.66      0.62      0.64      1953
      Kenny       0.97      0.96      0.96       157
       Kyle       0.45      0.45      0.45      1440
       Stan       0.49      0.51      0.50      1537

avg / total       0.56      0.55      0.55      5087

[[1218    2  377  356]
 [   1  150    4    2]
 [ 311    2  651  476]
 [ 323    0  426  788]]


In [146]:
# Опять же результаты у Кенни наиболее высокие - и precision, и recall близки к 100%. Немного лучше начали распознаваться 
# реплики Кайла, но не намного. Для него классификатор все равно хуже всего работает
# Средние показатели трех зарактеристик выросли на 2-3%, что немного, но все-таки хоть что-то.
# Интересно, что классификатор также ни одну реплику, принадлежащую Стэну, не приписал Кенни. 

In [147]:
# RANDOM FOREST

In [150]:
rfc = RandomForestClassifier(n_estimators=80)
rfc.fit(X_train, y_train)
y_pred_rf = rfc.predict(X_test)
print(classification_report(y_test, y_pred_rf))
print(confusion_matrix(y_test, y_pred_rf))

             precision    recall  f1-score   support

    Cartman       0.56      0.73      0.64      1953
      Kenny       0.99      0.94      0.97       157
       Kyle       0.45      0.31      0.37      1440
       Stan       0.46      0.42      0.44      1537

avg / total       0.51      0.52      0.51      5087

[[1427    0  222  304]
 [   6  148    3    0]
 [ 533    1  443  463]
 [ 570    0  315  652]]


In [154]:
# У Random Forest пока наихудшие результаты, хотя опять же не намного - на 1-2% хуже чем у наивного байеса.
# Лучше всего по прежнему классифицируются реплики Кенни. Хуже всего по прежнему Кайл.
# Ни одна реплика Стэна и Картмена не была классифицирована как реплика Кенни, и всего одна реплика Кайла попала к Кенни, 
# из-за чего у него такой высокий precision - 99%.
# Этот классификатор лучший из классификатор в правильном нахождении реплик Картмена.

In [ ]:
# RANDOM TREE

In [159]:
clf = DecisionTreeClassifier(min_samples_split=5)
clf.fit(X_train, y_train)
y_pred_clf = clf.predict(X_test)
print(classification_report(y_test, y_pred_clf))
print(confusion_matrix(y_test, y_pred_clf))

             precision    recall  f1-score   support

    Cartman       0.52      0.58      0.55      1953
      Kenny       0.99      0.94      0.97       157
       Kyle       0.40      0.35      0.37      1440
       Stan       0.41      0.41      0.41      1537

avg / total       0.47      0.48      0.47      5087

[[1134    0  357  462]
 [   6  148    1    2]
 [ 503    1  502  434]
 [ 520    0  382  635]]


In [ ]:
# Теперь у Random Tree хуже всего результаты. Так же как и вдругих классификаторах, лучше всего определялся Кенни, 
# хуже всего Кайл. Как и у Random Forest, здесь почти стопроцентная precision у Кенни, так как только одна реплика Кайла была
# неправильно приписана Кенни.